In [ ]:
import couchdb
import json
import pandas as pd 
import numpy as np
import os
from dotenv import load_dotenv
import re 
from collections import Counter
from IPython.display import JSON
import matplotlib.pyplot as plt
import seaborn as sns


## 1. Verbindung zur existierenden Datenbank

In [ ]:
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984"  
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try: 
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich verbunden mit CouchDB unter {COUCHDB_URL}")
    
    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank {DB_NAME} erfolgreich ausgewählt")
    else:
        print(f"Datenbank {DB_NAME} nicht gefunden!")
        raise LookupError(f"Datenbank {DB_NAME} nicht gefunden!")

except Exception as e:
    print(f"Fehler beim Verbinden mit CouchDB: {e}")
    raise e

### Hilfsfunktion zum Erstellen/Aktualisieren von Design-Dokumenten

In [ ]:
def sync_design_doc(db_handle, design_doc_dict):
    if not db_handle:
        print("sync_design_doc: Keine Datenbankverbindung.")
        return False
        
    doc_id = design_doc_dict['_id']
    try:
        existing_doc = db_handle.get(doc_id)
        if existing_doc:
            if existing_doc.get('views') != design_doc_dict.get('views') or \
               existing_doc.get('language') != design_doc_dict.get('language'):
                design_doc_dict['_rev'] = existing_doc['_rev'] 
                db_handle.save(design_doc_dict)
                print(f"Design-Dokument '{doc_id}' aktualisiert.")
                return True
            else:
                print(f"Design-Dokument '{doc_id}' ist bereits aktuell.")
                return False
        else:
            # Design-Dokument existiert nicht, neu erstellen
            db_handle.save(design_doc_dict)
            print(f"Design-Dokument '{doc_id}' erstellt.")
            return True
    except Exception as e:
        print(f"Fehler beim Synchronisieren des Design-Dokuments '{doc_id}': {e}")
        return False

### Hilfsfunktion zum Parsen von Zahlen aus Strings


In [ ]:
def parse_numeric_value_from_string(value_str, is_percentage=False, is_rate_per_1000=False):
    if pd.isna(value_str) or not isinstance(value_str, str):
        return np.nan # Wichtig für Pandas Operationen wie Quantilberechnung
    match = re.search(r'(-?\d+\.?\d*|-?\d*\.?\d+)', value_str)
    if match and match[0]:
        val = float(match.group(1))
        # Spezifische Anpassungen, falls nötig (hier nicht, da Quantile relativ sind)
        return val
    return np.nan

## Datenauswertung

## Analyse 1: Wirtschaftliche Dynamik und Ungleichheit 

**Fragestellung:** Welche Länder verzeichnen ein hohes reales Wirtschaftswachstum, weisen aber gleichzeitig eine hohe Einkommensungleichheit (gemessen am Gini-Koeffizienten) auf?

**Ziel der Analyse:** Diese Analyse soll Länder identifizieren, bei denen der generierte Wohlstand möglicherweise nicht breit in der Bevölkerung ankommt oder bei denen schnelles Wirtschaftswachstum mit einer Zunahme der Ungleichheit einhergeht. Dies kann Anstöße für Diskussionen über Verteilungsgerechtigkeit und die Nachhaltigkeit von Wachstumsmodellen geben.

**Benötigte Felder:**
* `Economy: Real GDP growth rate` – Gibt die jährliche prozentuale Veränderung des realen Bruttoinlandsprodukts an.
* `Economy: Gini Index coefficient - distribution of family income` – Ein Maß für die Ungleichverteilung der Einkommen oder des Konsums innerhalb eines Landes. Ein Wert von 0 bedeutet vollkommene Gleichheit, ein Wert von 100 bedeutet vollkommene Ungleichheit.
* `Government: Country name - conventional short form` – Für die Identifikation der Länder.

**CouchDB-Technik: Map View**
Erstellung einer Map View namens `gdp_growth_vs_gini`.
* **Map-Funktion:** Diese JavaScript-Funktion wird für jedes relevante Dokument ausgeführt.
    1.  Sie prüft, ob die Felder für die Wachstumsrate und den Gini-Index vorhanden sind.
    2.  Sie **parst** die als String vorliegenden Werte für Wachstumsrate (z.B. aus "3.5%" wird `3.5`) und den Gini-Koeffizienten (z.B. aus "42.5" wird `42.5`) in numerische Formate. 
    3.  Bei erfolgreichem Parsen emittiert die Funktion einen **zusammengesetzten Schlüssel** `[parsed_gdp_growth, parsed_gini]` und den Ländernamen als Wert.
    4.  Dokumente, bei denen das Parsing fehlschlägt oder Felder fehlen, werden nicht emittiert.

**Abfrage der View aus Python:**
Die erstellte View kann dann mit `startkey`- und `endkey`-Parametern abgefragt werden, um Länder zu finden, deren Wachstumsraten und Gini-Koeffizienten in bestimmten Wertebereichen liegen. Python dient dazu, die View-Definition zu erstellen/aktualisieren, die Abfrage zu formulieren und die Ergebnisse darzustellen.

In [ ]:
if db:
    design_doc_economic_analysis = {
        '_id': '_design/economic_analysis', 
        'language': 'javascript',
        'views': {
            'gdp_growth_vs_gini': { 
                'map': """
                    function(doc) {
                        if (doc.type === 'country' && 
                            doc["Economy: Real GDP growth rate"] && 
                            doc["Economy: Gini Index coefficient - distribution of family income"] &&
                            doc["Government: Country name - conventional short form"]) {

                            var gdp_growth_str = doc["Economy: Real GDP growth rate"];
                            var gini_str = doc["Economy: Gini Index coefficient - distribution of family income"];
                            var country_name = doc["Government: Country name - conventional short form"];

                            var parsed_gdp_growth = null;
                            var parsed_gini = null;

                            var match_growth = gdp_growth_str.match(/(-?\\d+\\.?\\d*)/);
                            if (match_growth[0]) {
                                parsed_gdp_growth = parseFloat(match_growth[0]);    
                            } 

                            var match_gini = gini_str.match(/(\\d+\\.?\\d*)/);
                            if (match_gini[0]) {
                                parsed_gini = parseFloat(match_gini[0]);
                            } 

                            emit([parsed_gdp_growth, parsed_gini], country_name);
                            
                        }
                    }
                """
            }
        }
    }

    # Design-Dokument in CouchDB speichern/aktualisieren
    sync_design_doc(db, design_doc_economic_analysis)

    wachstum_schwellenwert = 4.0
    gini_schwellenwert = 40.0

    print(f"\nView-Abfrage für Länder mit: \nReal GDP growth > {wachstum_schwellenwert}% UND Gini Index > {gini_schwellenwert}")
    try:
        query_startkey = [wachstum_schwellenwert, gini_schwellenwert]
        query_endkey = [{}, {}] # {} -> Bis zum Maximum für beide Schlüsselkomponenten

        results = db.view(
            'economic_analysis/gdp_growth_vs_gini',
            startkey=query_startkey,
            endkey=query_endkey,
            reduce=False 
        )

        found_countries = []
        for row in results:
            found_countries.append({
                'Land': row.value,
                'Wachstumsrate (%)': row.key[0],
                'Gini-Index': row.key[1]
            })

        if found_countries:
            df_results = pd.DataFrame(found_countries)
            print(f"\n{len(df_results)} Länder gefunden, die die Kriterien erfüllen:")
            display(df_results.sort_values(by=['Wachstumsrate (%)', 'Gini-Index'], ascending=[False, False]))
        else:
            print("\nKeine Länder gefunden, die die spezifizierten Kriterien erfüllen.")
            
    except Exception as e:
        print(f"Fehler beim Abfragen der View 'gdp_growth_vs_gini': {e}")
else:
    print("Keine Datenbankverbindung ('db'), Analyse übersprungen.")

## Analyse 2: Bildungsrendite-Paradoxon (Kategorisierte Profile)